# Imports

In [5]:
import time
from quasi_newton_method import quasi_newton_method_1D
from secant_method import secant_method_1D
import numpy as np
from Rosenbrok_Powell import rosenbrock_grad, powell_quartic_grad, powell_quartic_1d, powell_quartic_1d_grad, rosenbrock_1d, rosenbrock_1d_grad
import math
import sympy as sp

# Used in all problems

In [6]:
a = 0
b = 3
interval = [a,b]
EPSILON = 0.01
benchMarks = {}

# Fibonacci

## Initialization

In [7]:
Fs = [1,1]
maxFibonacciNumber = (interval[1]-interval[0])/EPSILON
while Fs[-1] < maxFibonacciNumber:
    Fs.append(Fs[-1] + Fs[-2])
Fs.pop()
Fs

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233]

## Setting Initial points

In [8]:
k = len(Fs)

x1 = a + (Fs[k-3] / Fs[k-1]) * (b - a)
x2 = a + (Fs[k-2] / Fs[k-1]) * (b - a)

ITERATIONS = k - 2

## Computing x1 and x2 in F(X)

In [9]:
fibonacciStartTime = time.time()
fibonacciIterations = 0
for i in range(ITERATIONS):
    fibonacciIterations+=1
    newInterval = []
    f_X1 = 2 * (x1)**2 - 5 * x1 + 3
    f_X2 = 2 * (x2)**2 - 5 * x2 + 3
    if f_X1 > f_X2:
        a = x1
        x1 = x2 
        x2 = a+ (Fs[k-2]/Fs[k-1]) * (b-a)
    else:
        b = x2
        x2 = x1
        x1 = a + (Fs[k-3]/Fs[k-1]) * (b-a)
    newInterval.append(round(a,3))
    newInterval.append(round(b,3))
    print(newInterval)
    k-=1
    if b-a <= EPSILON:
        
        break
fibonacciEndTime = time.time()

[0, 1.854]
[0.708, 1.854]
[0.708, 1.416]
[0.979, 1.416]
[1.146, 1.416]
[1.146, 1.313]
[1.21, 1.313]
[1.21, 1.273]
[1.233, 1.273]
[1.233, 1.257]
[1.241, 1.257]


## Results

In [10]:
finalInterval = []
finalInterval.append(round(a,3))
finalInterval.append(round(b,3))
print(finalInterval)
fibonacciMinimum = (a+b) / 2
print(round(fibonacciMinimum,3))
fibonacciF_min = 2 * (fibonacciMinimum)**2 - 5 * fibonacciMinimum + 3
print(round(fibonacciF_min,3))

[1.241, 1.257]
1.249
-0.125


## Benchmark

In [11]:
print(f"Converged after {fibonacciIterations} iterations!")
fibonacciTakenTime = fibonacciEndTime - fibonacciStartTime
print(f"Taken: {round(fibonacciTakenTime,9)} Seconds")

Converged after 11 iterations!
Taken: 0.0 Seconds


## Adding to benchmark to compare later

In [12]:
benchMarks["Fibonacci"] = {
                           "Iterations":fibonacciIterations,
                           "TimeTaken": round(fibonacciTakenTime, 6),
                           "F": round(fibonacciF_min, 3),
                           "X": round(fibonacciMinimum, 3)
                           }

# Golden Section

## Initialize interior Points

In [13]:
GOLDENRATIO = (math.sqrt(5)-1) / 2
x1 = a + (1-GOLDENRATIO) * (b - a)
x2 = a + GOLDENRATIO * (b - a)

## Substitute in function

In [14]:
f_X1 = 2 * (x1)**2 - 5 * x1 + 3
f_X2 = 2 * (x2)**2 - 5 * x2 + 3
goldenStartTime = time.time()
goldenIterations = 0
while (b - a) > EPSILON:
    goldenIterations+=1
    currentInterval = []
    if f_X1  < f_X2:
        b = x2
        x2 = x1
        x1 = a + (1-GOLDENRATIO) * (b-a)
    elif f_X1 > f_X2:
        a = x1
        x1 = x2
        x2 = a + GOLDENRATIO * (b-a)
    f_X1 = 2 * (x1)**2 - 5 * x1 + 3
    f_X2 = 2 * (x2)**2 - 5 * x2 + 3
    currentInterval.append(round(a,3))
    currentInterval.append(round(b,3))
    print(currentInterval)

goldenEndTime = time.time()


[1.247, 1.257]


## Results

In [15]:
goldenMinimum = (a+b) / 2
print(round(goldenMinimum,3))
goldenF_min = 2 * (goldenMinimum)**2 - 5 * goldenMinimum + 3
print(round(goldenF_min,3))

1.252
-0.125


## Benchmark

In [16]:
print(f"Converged after {goldenIterations} iterations")
goldenTakenTime = goldenEndTime - goldenStartTime
print(f"Taken: {round(goldenTakenTime,9)} Seconds")

Converged after 1 iterations
Taken: 0.0 Seconds


## Adding to benchmark to compare later

In [17]:
benchMarks["Golden"] = {
                        "Iterations":goldenIterations,
                        "TimeTaken": round(goldenTakenTime, 6),
                        "F": round(goldenF_min, 3),
                        "X": round(goldenMinimum, 3)
                        }


# Newton

## Initial X and Computing 1st and 2nd derivatives

In [18]:
xSymbol = sp.symbols('x')
newtonF_x = 2 * (xSymbol)**2 - 5 * xSymbol + 3
dF_x_1 = sp.diff(newtonF_x, xSymbol)
dF_x_2 = sp.diff(dF_x_1, xSymbol)

## Updating x

In [19]:
newtonX = 2
newtonOldX = 0
newtonIterations = 0
# x = x - (dF_x_1/dF_x_2)
newtonStartTime = time.time()
while abs(newtonX-newtonOldX) > EPSILON:
    newtonIterations+=1
    dF_x_1_atx = dF_x_1.subs(xSymbol,newtonX)
    dF_x_2_atx = dF_x_2.subs(xSymbol,newtonX)
    newtonOldX = newtonX
    newtonX = float(newtonX - (dF_x_1_atx/dF_x_2_atx))
    print(newtonX)
    if newtonX-newtonOldX < EPSILON:
        
        break
    
newtonEndTime = time.time()

1.25


## Results

In [20]:
print(newtonX)
newtonF_min = 2*(newtonX)**2 -5 * newtonX +3
print(newtonF_min)

1.25
-0.125


## Benchmark

In [21]:
print(f"Converged after {newtonIterations} iterations")
newtonTakenTime = newtonEndTime - newtonStartTime
print(f"Taken: {newtonTakenTime} Seconds")

Converged after 1 iterations
Taken: 0.0010020732879638672 Seconds


## Adding to benchmark to compare later

In [22]:
benchMarks["Newton"] = {
                        "Iterations":newtonIterations,
                        "TimeTaken": round(newtonTakenTime, 6),
                        "F": round(newtonF_min, 3),
                        "X": round(newtonX, 3)
                        }

# Benchmarks for first three

In [23]:
benchMarks

{'Fibonacci': {'Iterations': 11, 'TimeTaken': 0.0, 'F': -0.125, 'X': 1.249},
 'Golden': {'Iterations': 1, 'TimeTaken': 0.0, 'F': -0.125, 'X': 1.252},
 'Newton': {'Iterations': 1, 'TimeTaken': 0.001002, 'F': -0.125, 'X': 1.25}}

# Part 2 - Quasi-Newton and Secant Methods

# Quasi-Newton Method (1D)

In [24]:
def quasi_newton_method_1D(f, f_prime, x0, tol=1e-5, max_iter=1000):
    """Finds the minimizer using Quasi-Newton method (1D)."""
    x = x0
    iterations = 0
    start_time = time.time()
    h = 1e-5
    # Approximate Hessian using finite difference
    hessian = (f_prime(x + h) - f_prime(x))/h

    while abs(f_prime(x)) > tol and iterations < max_iter:
        x = x - f_prime(x) / hessian  # Update x based on approximated Hessian
        iterations += 1
    end_time = time.time()
    cpu_time = end_time - start_time
    return x, f(x), iterations, cpu_time

# Secant Method (1D)

In [25]:
def secant_method_1D(f, x0, x1, tol=1e-5, max_iter=1000):
    """Finds the minimizer using the Secant method (1D)."""
    start_time = time.time()
    x_prev = x0
    x_curr = x1
    iterations = 0
    while abs(f(x_curr) - f(x_prev)) > tol and iterations < max_iter:

        x_next = x_curr - f(x_curr) * (x_curr - x_prev) / \
            (f(x_curr) - f(x_prev))  # Secant formula
        x_prev = x_curr
        x_curr = x_next
        iterations += 1
    end_time = time.time()
    cpu_time = end_time - start_time
    return x_curr, f(x_curr), iterations, cpu_time

# Benchmark Functions from Document

### Rosenbrock's function and its gradient

In [26]:
def rosenbrock(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2


def rosenbrock_grad(x):
    return np.array([
        -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0]),
        200 * (x[1] - x[0]**2)
    ])

### Powell's quartic function

In [27]:
def powell_quartic(x):
    return (x[0] + 10 * x[1])**2 + 5 * (x[2] - x[3])**2 + (x[1] - 2 * x[2])**4 + 10 * (x[0] - x[3])**4

# For Powell's quartic, finding the gradient analytically is complex; let's use finite differences


def powell_quartic_grad(x, h=1e-5):
    grad = np.zeros_like(x)
    for i in range(len(x)):
        x_plus_h = x.copy()
        x_plus_h[i] += h
        grad[i] = (powell_quartic(x_plus_h) - powell_quartic(x)) / h
    return grad

### Function for 1D minimization with Line Search using Rosenbrock Function

In [28]:
def rosenbrock_1d(x, d, alpha):
    x_new = x + alpha*d
    return rosenbrock(x_new)


def rosenbrock_1d_grad(x, d, alpha):
    h = 1e-5
    return (rosenbrock_1d(x, d, alpha+h)-rosenbrock_1d(x, d, alpha))/h

### Function for 1D minimization with Line Search using Powell's Function

In [29]:
def powell_quartic_1d(x, d, alpha):
    x_new = x + alpha*d
    return powell_quartic(x_new)


def powell_quartic_1d_grad(x, d, alpha):
    h = 1e-5
    return (powell_quartic_1d(x, d, alpha+h)-powell_quartic_1d(x, d, alpha))/h

In [30]:
# Initial guess for Rosenbrock
x0_rosen = np.array([-1.2, 1.0])

# For 1D methods we need to perform Line Search
# We need to find an descent direction vector d for 1D minimization
# In this example, I will use d = -grad(x)
d_rosen = -rosenbrock_grad(x0_rosen)

# Let's run the methods for Line Search
initial_alpha = 0.0
initial_alpha_secant = 0.1

print("\n-- Quasi-Newton Method --")
min_alpha, min_val, iterations, cpu_time = quasi_newton_method_1D(lambda alpha: rosenbrock_1d(x0_rosen, d_rosen, alpha),
                                                                  lambda alpha: rosenbrock_1d_grad(
                                                                      x0_rosen, d_rosen, alpha),
                                                                  initial_alpha)

print(f"Optimal alpha: {min_alpha}, f(x): {
      min_val}, iterations: {iterations}, CPU time: {cpu_time}")

print("\n-- Secant Method --")
min_alpha, min_val, iterations, cpu_time = secant_method_1D(lambda alpha: rosenbrock_1d(x0_rosen, d_rosen, alpha),
                                                            initial_alpha, initial_alpha_secant)
print(f"Optimal alpha: {min_alpha}, f(x): {
      min_val}, iterations: {iterations}, CPU time: {cpu_time}")

# --- Powell's Quartic Function ---
print("\n--- Powell's Quartic Function ---")

# Initial guess for Powell's quartic
x0_powell = np.array([3.0, -1.0, 0.0, 1.0])

# For 1D methods we need to perform Line Search
# We need to find an descent direction vector d for 1D minimization
# In this example, I will use d = -grad(x)
d_powell = -powell_quartic_grad(x0_powell)

# Let's run the methods for Line Search
initial_alpha = 0.0
initial_alpha_secant = 0.1

print("\n-- Quasi-Newton Method --")
min_alpha, min_val, iterations, cpu_time = quasi_newton_method_1D(lambda alpha: powell_quartic_1d(x0_powell, d_powell, alpha),
                                                                  lambda alpha: powell_quartic_1d_grad(
                                                                      x0_powell, d_powell, alpha),
                                                                  initial_alpha)
print(f"Optimal alpha: {min_alpha}, f(x): {
      min_val}, iterations: {iterations}, CPU time: {cpu_time}")

print("\n-- Secant Method --")
min_alpha, min_val, iterations, cpu_time = secant_method_1D(lambda alpha: powell_quartic_1d(x0_powell, d_powell, alpha),
                                                            initial_alpha, initial_alpha_secant)

print(f"Optimal alpha: {min_alpha}, f(x): {
      min_val}, iterations: {iterations}, CPU time: {cpu_time}")

# --- Benchmarks for all ---
print("\n# --- Benchmarks for all Methods ---")
print(benchMarks)


-- Quasi-Newton Method --
Optimal alpha: 0.0007830046366470301, f(x): 4.128804567900709, iterations: 18, CPU time: 0.0010006427764892578

-- Secant Method --
Optimal alpha: 0.012261706107240849, f(x): 0.19932179161009153, iterations: 400, CPU time: 0.005633115768432617

--- Powell's Quartic Function ---

-- Quasi-Newton Method --
Optimal alpha: 0.003583798955555407, f(x): 30.83034739482649, iterations: 693, CPU time: 0.01974010467529297

-- Secant Method --
Optimal alpha: 0.0035764060670935975, f(x): 30.830704344591066, iterations: 76, CPU time: 0.0010340213775634766

# --- Benchmarks for all Methods ---
{'Fibonacci': {'Iterations': 11, 'TimeTaken': 0.0, 'F': -0.125, 'X': 1.249}, 'Golden': {'Iterations': 1, 'TimeTaken': 0.0, 'F': -0.125, 'X': 1.252}, 'Newton': {'Iterations': 1, 'TimeTaken': 0.001002, 'F': -0.125, 'X': 1.25}}
